In [ ]:
import Pkg
Pkg.activate("..")
Pkg.instantiate()

using RadiiPolynomial
using LinearAlgebra
using MarchalConjecture

BLAS.set_num_threads(8) # change the number of threads accordingly

In [ ]:
# initialize data: triangle

K = 70 # truncation order for the Fourier series

# symmetric space

space_U_ = EvenCos(K, 1.0) × EvenSin(K, 1.0) × OddCos(K, 1.0)
space_V_ = EvenSin(K, 1.0) × EvenCos(K, 1.0) × OddSin(K, 1.0)
space_W_ = EvenFourier(K, 1.0)

space_X_ = CartesianProduct(ParameterSpace(), ParameterSpace(), ParameterSpace(), space_U_, space_V_, space_W_)

x_full_ = zeros(ComplexF64, desymmetrize(space_X_))

component(component(x_full_, 4), 1)[-2] =       3^(-1/6) / 2
component(component(x_full_, 4), 1)[ 2] =       3^(-1/6) / 2
component(component(x_full_, 4), 2)[-2] =  im * 3^(-1/6) / 2
component(component(x_full_, 4), 2)[ 2] = -im * 3^(-1/6) / 2
component(component(x_full_, 4), 3)[-1] = 1 / 2
component(component(x_full_, 4), 3)[ 1] = 1 / 2

component(component(x_full_, 5), 1)[-2] = -im * 3^(-1/6)
component(component(x_full_, 5), 1)[ 2] =  im * 3^(-1/6)
component(component(x_full_, 5), 2)[-2] =       3^(-1/6)
component(component(x_full_, 5), 2)[ 2] =       3^(-1/6)
component(component(x_full_, 5), 3)[-1] = -im / 2
component(component(x_full_, 5), 3)[ 1] =  im / 2

component(x_full_, 6)[0] = 3^(-1/3)

x_ = project_sym!(zeros(ComplexF64, space_X_), x_full_)

# continuation

N = 20 # truncation order for the Chebyshev series
npts = N + 1 # number of points for DFT

Ω_grid_ = [0.5 + 0.5cospi(j/N) for j ∈ 0:npts-1] # Chebyshev nodes in the interval [0, 1]
x_grid_ = Vector{typeof(x_)}(undef, npts)

# to be more memory efficient
_cache_F_       = zeros(ComplexF64, space_X_)
_cache_DF_      = zeros(ComplexF64, space_X_, space_X_)
_cache_x_full_  = zeros(ComplexF64, desymmetrize(space_X_))
_cache_F_full_  = zeros(ComplexF64, desymmetrize(space_X_))
_cache_DF_full_ = zeros(ComplexF64, desymmetrize(space_X_), desymmetrize(space_X_))
#

for j ∈ 1:npts
    x_grid_[j] = j == 1 ? copy(x_) : copy(x_grid_[j-1])

    _, success = newton!(x_grid_[j], _cache_F_, _cache_DF_; tol = 1e-15, verbose = true) do F, DF, x
        _cache_x_full_ .= 0
        F .= 0
        DF .= 0
        project_sym!(_cache_x_full_, x)

        F_marchal!(_cache_F_full_, _cache_x_full_, Ω_grid_[j])
        DF_marchal!(_cache_DF_full_, _cache_x_full_, Ω_grid_[j])

        project_sym!(F, _cache_F_full_)
        project_sym!(DF, _cache_DF_full_)

        return F, DF
    end
    success && println("Success: Newton's method converged at the Chebyshev node Ωⱼ = $(Ω_grid_[j])\n")
    success || error()
end

In [ ]:
using GLMakie

x_plot = map(x -> project_sym!(zeros(ComplexF64, desymmetrize(space_X_)), x), x_grid_)

fig = Figure()

ax = Axis3(fig[1,1]; aspect = :data, viewmode = :fitzoom, elevation = π/10, perspectiveness = 0.075,
    xlabel = L"U_x", ylabel = L"U_y", zlabel = L"U_z")

cmap = range(colorant"royalblue1"; stop = colorant"salmon", length = npts)

for j ∈ 1:npts
    amplitude = sqrt(abs(real(x_plot[j][1])))
    lines!(ax,
        [Point3f(real(component(component(x_plot[j], 4), 1)(t)),
                 real(component(component(x_plot[j], 4), 2)(t)),
                 amplitude * real(component(component(x_plot[j], 4), 3)(t))) for t ∈ LinRange(0, 2π, 1_000)];
        color = cmap[j])
end

fig

In [ ]:
space_U = EvenCos(K, interval(1)) × EvenSin(K, interval(1)) × OddCos(K, interval(1))
space_V = EvenSin(K, interval(1)) × EvenCos(K, interval(1)) × OddSin(K, interval(1))
space_W = EvenFourier(K, interval(1))

space_X = CartesianProduct(ParameterSpace(), ParameterSpace(), ParameterSpace(), space_U, space_V, space_W)

space_X_full_ = ParameterSpace() × ParameterSpace() × ParameterSpace() × Fourier(K, 1)^3 × Fourier(K, 1)^3 × Fourier(K, 1)

x_full_grid_ = map(xⱼ -> project_sym!(zeros(ComplexF64, space_X_full_), xⱼ), x_grid_)

space_X_full = ParameterSpace() × ParameterSpace() × ParameterSpace() × Fourier(K, interval(1))^3 × Fourier(K, interval(1))^3 × Fourier(K, interval(1))

x_full_grid = map(xⱼ_full -> Sequence(space_X_full, interval.(coefficients(xⱼ_full))), x_full_grid_)


# clean up the numerics

conjugacy_symmetry!.(x_full_grid)
for j ∈ 1:npts
    x_full_grid[j][2] = interval(0) # set β = 0
    x_full_grid[j][3] = interval(0) # set α = 0
    for k ∈ -K:K
        component(component(x_full_grid[j], 4), 1)[k] = real(component(component(x_full_grid[j], 4), 1)[k])
        component(component(x_full_grid[j], 4), 2)[k] = interval(im) * imag(component(component(x_full_grid[j], 4), 2)[k])
        component(component(x_full_grid[j], 4), 3)[k] = real(component(component(x_full_grid[j], 4), 3)[k])

        component(component(x_full_grid[j], 5), 1)[k] = interval(im) * imag(component(component(x_full_grid[j], 5), 1)[k])
        component(component(x_full_grid[j], 5), 2)[k] = real(component(component(x_full_grid[j], 5), 2)[k])
        component(component(x_full_grid[j], 5), 3)[k] = interval(im) * imag(component(component(x_full_grid[j], 5), 3)[k])
    end
end


# at Ω = 1, the periodic orbit is the Lagrange triangle

x_full_grid[1] .= interval(0) # we first reset all components to zero

component(component(x_full_grid[1], 4), 1)[-2] =                 inv(sqrt(cbrt(interval(3)))) / interval(2)
component(component(x_full_grid[1], 4), 1)[ 2] =                 inv(sqrt(cbrt(interval(3)))) / interval(2)
component(component(x_full_grid[1], 4), 2)[-2] = interval( im) * inv(sqrt(cbrt(interval(3)))) / interval(2)
component(component(x_full_grid[1], 4), 2)[ 2] = interval(-im) * inv(sqrt(cbrt(interval(3)))) / interval(2)
component(component(x_full_grid[1], 4), 3)[-1] = interval(1) / interval(2)
component(component(x_full_grid[1], 4), 3)[ 1] = interval(1) / interval(2)

component(component(x_full_grid[1], 5), 1)[-2] = interval(-im) * inv(sqrt(cbrt(interval(3))))
component(component(x_full_grid[1], 5), 1)[ 2] = interval( im) * inv(sqrt(cbrt(interval(3))))
component(component(x_full_grid[1], 5), 2)[-2] =                 inv(sqrt(cbrt(interval(3))))
component(component(x_full_grid[1], 5), 2)[ 2] =                 inv(sqrt(cbrt(interval(3))))
component(component(x_full_grid[1], 5), 3)[-1] = interval(-im) * interval(1) / interval(2)
component(component(x_full_grid[1], 5), 3)[ 1] = interval( im) * interval(1) / interval(2)

component(x_full_grid[1], 6)[0] = inv(cbrt(interval(3)))


# at Ω = 0, the periodic orbit is the figure eight (here we only impose that the solution lies in the yz-plane)
# note: the proof of the eight-shape figure (i.e. Lemma 2.2 (iv)) is done a posteriori

component(component(x_full_grid[npts], 4), 1) .= interval(0)
component(component(x_full_grid[npts], 5), 1) .= interval(0)

nothing # do not print output

In [ ]:
# proof of the existence of a zero

ν = I"1.1"


# use DFT to get the candidate branch

big_x_full = dft_grid2cheb(map(xⱼ -> setprecision(() -> big.(xⱼ), BigFloat, 256), x_full_grid))
x_full = Sequence(space(big_x_full), map(xⱼ -> interval.(Float64, xⱼ), coefficients(big_x_full)))


# impose symmetries and complex conjugacy

conjugacy_symmetry!(x_full)
x_full[1][:] .= real.(x_full[1][:])
x_full[2][:] .= interval(0) # set β = 0
x_full[3][:] .= interval(0) # set α = 0
for k ∈ -K:K
    component(component(x_full, 4), 1)[k][:] .= real.(component(component(x_full, 4), 1)[k][:])
    component(component(x_full, 4), 2)[k][:] .= complex.(interval(0), imag.(component(component(x_full, 4), 2)[k][:]))
    component(component(x_full, 4), 3)[k][:] .= real.(component(component(x_full, 4), 3)[k][:])

    component(component(x_full, 5), 1)[k][:] .= complex.(interval(0), imag.(component(component(x_full, 5), 1)[k][:]))
    component(component(x_full, 5), 2)[k][:] .= real.(component(component(x_full, 5), 2)[k][:])
    component(component(x_full, 5), 3)[k][:] .= complex.(interval(0), imag.(component(component(x_full, 5), 3)[k][:]))
end
#

Ω = Sequence(Chebyshev(1) ⊗ Fourier(0, interval(1)), [inv(interval(2)), -inv(interval(4))])

space_X_full_tensor =
    (Chebyshev(N) ⊗ Fourier(0, interval(1)))   × (Chebyshev(N) ⊗ Fourier(0, interval(1)))   × (Chebyshev(N) ⊗ Fourier(0, interval(1))) ×
    (Chebyshev(N) ⊗ Fourier(K, interval(1)))^3 × (Chebyshev(N) ⊗ Fourier(K, interval(1)))^3 × (Chebyshev(N) ⊗ Fourier(K, interval(1)))
x_full_tensor = Sequence(space_X_full_tensor, mapreduce(coefficients, vcat, x_full))


# construct the finite part of A (called A_finite in the paper)

A_grid_ = map(x_full_grid_, Ω_grid_) do xⱼ_full, Ωⱼ
    return inv(project_sym!(_cache_DF_, DF_marchal!(_cache_DF_full_, xⱼ_full, Ωⱼ)))
end

A_ = dft_grid2cheb(A_grid_)
A = LinearOperator(domain(A_), codomain(A_), map(A_ⱼ -> interval.(A_ⱼ), coefficients(A_)))

opnorm_A = opnorm1_sym(norm.(A, 1), ν)

### Lemma 3.2

In [6]:
# Chebyshev polynomial of order 7N

N7_fft = nextpow(2, 2*(7N)+1)
npts_N7 = N7_fft ÷ 2 + 1

Ω_grid_N7 = [inv(interval(2)) + cospi( interval(2j)/interval(N7_fft) ) / interval(2) for j ∈ 0:npts_N7-1]

big_x_full_grid_N7 = cheb2grid(big_x_full, N7_fft)
x_full_grid_N7 = map(xⱼ -> interval.(Float64, xⱼ), big_x_full_grid_N7)

A_grid_N7 = cheb2grid(A, N7_fft)

#-- finite part

space_F_4 = EvenCos(4K, interval(1)) × EvenSin(4K, interval(1)) × OddCos(4K, interval(1))
space_F_5 = EvenSin( K, interval(1)) × EvenCos( K, interval(1)) × OddSin( K, interval(1))
space_F_6 = EvenFourier(5K, interval(1))
space_F = CartesianProduct(ParameterSpace(), ParameterSpace(), ParameterSpace(), space_F_4, space_F_5, space_F_6)

F_grid_N7 = map(x_full_grid_N7, Ω_grid_N7) do xⱼ_full, Ωⱼ
    Fⱼ_full = zeros(Complex{Interval{Float64}}, desymmetrize(space_F))
    F_marchal!(Fⱼ_full, xⱼ_full, Ωⱼ)
    return project_sym!(zeros(Complex{Interval{Float64}}, space_F), Fⱼ_full)
end

AF_grid_N7 = map((Aⱼ, Fⱼ) -> Sequence(space_X, coefficients(LinearOperator(coefficients(Aⱼ)) * Sequence(coefficients(project(Fⱼ, space_X))))), A_grid_N7, F_grid_N7)
AF = grid2cheb(AF_grid_N7, 7N)

#-- infinite part

F = grid2cheb(F_grid_N7, 6N)
tail_F = copy(component(F, [4,6]))
tail_F_j = component(tail_F, 1)
for i ∈ 1:nspaces(space(tail_F_j))
    tail_F_ji = component(tail_F_j, i)
    for k ∈ indices(space(tail_F_ji))
        if abs(k) ≤ K
            view(tail_F_ji[k], :) .= interval(0)
        end
    end
end
tail_F_j = component(tail_F, 2)
for k ∈ indices(space(tail_F_j))
    if abs(k) ≤ K
        view(tail_F_j[k], :) .= interval(0)
    end
end

#--

bounds(norm1_sym(norm.(AF, 1), ν) + norm1_sym(norm.(tail_F, 1), ν) / interval(K + 1))

### Lemma 3.3

In [ ]:
# Chebyshev polynomial of order 6N

N6_fft = nextpow(2, 2*(6N)+1) # 2N6_fft == N7_fft
npts_N6 = N6_fft ÷ 2 + 1

Ω_grid_N6 = Ω_grid_N7[1:2:end] # [inv(interval(2)) + cospi( interval(2j)/interval(N6_fft) ) / interval(2) for j ∈ 0:npts_N6-1]

x_full_grid_N6 = x_full_grid_N7[1:2:end] # cheb2grid(x_full, N6_fft)

A_grid_N6 = A_grid_N7[1:2:end] # cheb2grid(A, N6_fft)

#--

domain_DF_4 = EvenCos(2K, interval(1)) × EvenSin(2K, interval(1)) × OddCos(2K, interval(1))
domain_DF_5 = EvenSin(2K, interval(1)) × EvenCos(2K, interval(1)) × OddSin(2K, interval(1))
domain_DF_6 = EvenFourier(2K, interval(1))
domain_DF = CartesianProduct(ParameterSpace(), ParameterSpace(), ParameterSpace(), domain_DF_4, domain_DF_5, domain_DF_6)
codomain_DF = space_X

DF_grid_N6 = map(x_full_grid_N6, Ω_grid_N6) do xⱼ_full, Ωⱼ
    DFⱼ_full = zeros(Complex{Interval{Float64}}, desymmetrize(domain_DF), desymmetrize(codomain_DF))
    DF_marchal!(DFⱼ_full, xⱼ_full, Ωⱼ)
    return project_sym!(zeros(Complex{Interval{Float64}}, domain_DF, codomain_DF), DFⱼ_full)
end

ADF_I_grid_N6 = map((Aⱼ, DFⱼ) -> LinearOperator(domain_DF, codomain_DF, coefficients(LinearOperator(coefficients(Aⱼ)) * LinearOperator(coefficients(DFⱼ)))) - UniformScaling(interval(1)), A_grid_N6, DF_grid_N6)
ADF_I = grid2cheb(ADF_I_grid_N6, 6N)

#---

ζ = interval(2)*interval(π)/interval(3)


a = component(x_full_tensor, 1)
u = component(x_full_tensor, 4)
ux, uy, uz = eachcomponent(u)

v = component(x_full_tensor, 5)
vx, vy, vz = eachcomponent(v)

w = component(x_full_tensor, 6)


Δux1 = ux - shift(ux, (0, interval(2)*ζ))
Δuy1 = uy - shift(uy, (0, interval(2)*ζ))
Δuz1 = uz - shift(uz, (0, interval(2)*ζ))

Δux2 = ux - shift(ux, (0, interval(4)*ζ))
Δuy2 = uy - shift(uy, (0, interval(4)*ζ))
Δuz2 = uz - shift(uz, (0, interval(4)*ζ))

Δvx1 = vx - shift(vx, (0, interval(2)*ζ))
Δvy1 = vy - shift(vy, (0, interval(2)*ζ))
Δvz1 = vz - shift(vz, (0, interval(2)*ζ))

X = Ell1(IdentityWeight(), GeometricWeight(ν))

w² = w * w

w³ = w² * w

w³Δvx1  =     w³ * Δvx1
w³Δvy1  =     w³ * Δvy1
aw³Δvz1 = a * w³ * Δvz1

w³Δux1  =     w³ * Δux1
w³Δuy1  =     w³ * Δuy1
aw³Δuz1 = a * w³ * Δuz1

w²3Δux1 = interval(3) * w² * Δux1
w²3Δuy1 = interval(3) * w² * Δuy1
w²3Δuz1 = interval(3) * w² * Δuz1

w²3Δux2 = interval(3) * w² * Δux2
w²3Δuy2 = interval(3) * w² * Δuy2
w²3Δuz2 = interval(3) * w² * Δuz2

w²3LaΔuΔv = interval(3) * w² * (Δux1 * Δvx1 + Δuy1 * Δvy1 + a * Δuz1 * Δvz1)

M_1 = max(
    norm(Ω^2, X) + sqrt(interval(3)) * (interval(2) * norm(w³, X) + norm( w³Δvx1, X)),
    norm(Ω^2, X) + sqrt(interval(3)) * (interval(2) * norm(w³, X) + norm( w³Δvy1, X)),
                   sqrt(interval(3)) * (interval(2) * norm(w³, X) + norm(aw³Δvz1, X)),
    interval(1) + norm(interval(2) * Ω, X) + sqrt(interval(3)) * norm( w³Δux1, X),
    interval(1) + norm(interval(2) * Ω, X) + sqrt(interval(3)) * norm( w³Δuy1, X),
    interval(1) +                            sqrt(interval(3)) * norm(aw³Δuz1, X),
    norm(w²3Δux1, X) + norm(w²3Δuy1, X) + norm(w²3Δuz1, X) +
        norm(w²3Δux2, X) + norm(w²3Δuy2, X) + norm(w²3Δuz2, X) +
        norm(w²3LaΔuΔv, X)
    )

tail_w³ = get_tail(w³, K)

tail_w³Δvx1  = get_tail( w³Δvx1, K)
tail_w³Δvy1  = get_tail( w³Δvy1, K)
tail_aw³Δvz1 = get_tail(aw³Δvz1, K)

tail_w³Δux1  = get_tail( w³Δux1, K)
tail_w³Δuy1  = get_tail( w³Δuy1, K)
tail_aw³Δuz1 = get_tail(aw³Δuz1, K)

tail_w²3Δux1 = get_tail(w²3Δux1, K)
tail_w²3Δuy1 = get_tail(w²3Δuy1, K)
tail_w²3Δuz1 = get_tail(w²3Δuz1, K)

tail_w²3Δux2 = get_tail(w²3Δux2, K)
tail_w²3Δuy2 = get_tail(w²3Δuy2, K)
tail_w²3Δuz2 = get_tail(w²3Δuz2, K)

tail_w²3LaΔuΔv = get_tail(w²3LaΔuΔv, K)

Δux1_0 = Δux1(nothing, interval(0))
Δuy1_0 = Δuy1(nothing, interval(0))
Δuz1_0 = Δuz1(nothing, interval(0))

M_2 = max(
    sqrt(interval(3)) * (               interval(2) * norm(w(nothing, interval(0))^2 * Δux1(nothing, interval(0)), X)  / ν^interval(2K+1) + interval(2) * norm(tail_w³, X) + norm(tail_w³Δvx1, X)),
    sqrt(interval(3)) * (               interval(2) * norm(w(nothing, interval(0))^2 * Δuy1(nothing, interval(0)), X)  / ν^interval(2K+1) + interval(2) * norm(tail_w³, X) + norm(tail_w³Δvy1, X)),
    sqrt(interval(3)) * ((interval(1) + interval(2) * norm(w(nothing, interval(0))^2 * Δuz1(nothing, interval(0)), X)) / ν^interval(2K+1) + interval(2) * norm(tail_w³, X) + norm(tail_aw³Δvz1, X)),
    sqrt(interval(3)) * norm(tail_w³Δux1, X),
    sqrt(interval(3)) * norm(tail_w³Δuy1, X),
    sqrt(interval(3)) * norm(tail_aw³Δuz1, X),
    interval(2) * norm(w(nothing, interval(0)) * (Δux1_0 * Δux1_0 + Δuy1_0 * Δuy1_0 + a * Δuz1_0 * Δuz1_0), X) / ν^interval(2K+1) +
    norm(tail_w²3Δux1, X) + norm(tail_w²3Δuy1, X) + norm(tail_w²3Δuz1, X) +
        norm(tail_w²3Δux2, X) + norm(tail_w²3Δuy2, X) + norm(tail_w²3Δuz2, X) +
        norm(tail_w²3LaΔuΔv, X)
    )

#---

bounds(max(
    opnorm1_sym(norm.(ADF_I, 1), ν) + max(norm(get_tail(w³ * Δuz1 * Δvz1, K), X), M_1) / interval(K + 1),
    opnorm_A * M_2 + M_1 / interval(K + 1)
    ))

### Lemma 3.4

In [ ]:
r = I"1e-6"
â  = norm(a,  X) + r
ûx = norm(ux, X) + r
ûy = norm(uy, X) + r
ûz = norm(uz, X) + r
v̂x = norm(vx, X) + r
v̂y = norm(vy, X) + r
v̂z = norm(vz, X) + r
ŵ  = norm(w,  X) + r

opnorm_D²F =
    max(
        interval(6) * max(ŵ^2 * ûz, ŵ * ûz^2) + interval(3) * max(ŵ^3 * v̂z, ŵ^3 * ûz, interval(3) * ŵ^2 * ûz * v̂z),
        interval(6) * max(              ŵ^2, interval(2) * ŵ * ûx)     + interval(6) * sqrt(interval(3)) * ŵ^2 + interval(3) * max(              ŵ^3, interval(3) * ŵ^2 * v̂x),
        interval(6) * max(              ŵ^2, interval(2) * ŵ * ûy)     + interval(6) * sqrt(interval(3)) * ŵ^2 + interval(3) * max(              ŵ^3, interval(3) * ŵ^2 * v̂y),
        interval(6) * max(ŵ^2 * ûz, â * ŵ^2, interval(2) * â * ŵ * ûz) + interval(6) * sqrt(interval(3)) * ŵ^2 + interval(3) * max(ŵ^3 * v̂z, â * ŵ^3, interval(3) * â * ŵ^2 * v̂z),
                                                                                                    interval(3) * max(              ŵ^3, interval(3) * ŵ^2 * ûx),
                                                                                                    interval(3) * max(              ŵ^3, interval(3) * ŵ^2 * ûy),
                                                                                                    interval(3) * max(ŵ^3 * ûz, â * ŵ^3, interval(3) * â * ŵ^2 * ûz),
        interval(6) * max(ŵ * ûz^2, interval(2) * ŵ * ûx, interval(2) * ŵ * ûy, interval(2) * ŵ * â * ûz, ûx^2 + ûy^2 + â * ûz^2) +
            interval(6) * sqrt(interval(3)) * (max(ŵ^2, interval(2) * ŵ * ûx) + max(ŵ^2, interval(2) * ŵ * ûy) + max(ŵ^2, interval(2) * ŵ * ûz)) +
            interval(9) * max(ŵ^2 * ûz * v̂z,
                              ŵ^2 * v̂x, ŵ^2 * v̂y, ŵ^2 * â * v̂z,
                              ŵ^2 * ûx, ŵ^2 * ûy, ŵ^2 * â * ûz,
                              interval(2) * ŵ * (ûx * v̂x + ûy * v̂y + ûz * v̂z * â),)
        )

bounds(max(opnorm_A, inv(interval(K + 1))) * opnorm_D²F)

### Proof of the eight-shape figure

We verify that the angular momentum does not vanish between $(0, \pi/2)$ (see Point (iv) of Lemma 2.2).

In [ ]:
Uy = component(component(x_full_grid[npts], 4), 2)
Uz = sqrt(real(x_full_grid[npts][1])) * component(component(x_full_grid[npts], 4), 3)

Vy = component(component(x_full_grid[npts], 5), 2)
Vz = sqrt(real(x_full_grid[npts][1])) * component(component(x_full_grid[npts], 5), 3)

Uy_valid = ValidatedSequence(Uy, r, Ell1(GeometricWeight(ν)))
Uz_valid = ValidatedSequence(Uz, r, Ell1(GeometricWeight(ν)))
Vy_valid = ValidatedSequence(Vy, r, Ell1(GeometricWeight(ν)))
Vz_valid = ValidatedSequence(Vz, r, Ell1(GeometricWeight(ν)))

#

μ = Uy_valid * Vz_valid - Uz_valid * Vy_valid

check = false

# prove that μ is negative on [0, 3/2]

for i ∈ mince(interval(0, 1.5), 2_000)
    check = sup(real( μ(i) )) < 0
    check || break
end
check && println("Success: μ is negative on [0, 3/2]")

# prove that μ is positive on [3/2, π/2]

∂³μ = differentiate(μ, 3)

for i ∈ mince(interval(1.5, sup(interval(π)/interval(2))), 100)
    check = inf(real( ∂³μ(i) )) > 0
    check || break
end
check && println("Success: μ is positive on [3/2, π/2]")